In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import nibabel as nib
import os
import time

import pandas as pd
import numpy as np

from mricode.utils import log_textfile

from mricode.utils import return_csv
#from mricode.utils import return_iter

In [3]:
path_output = './'
path_tfrecords = '/data2/res64/down/'
path_csv = '/data2/csv/'
filename_res = {'train': 'intell_residual_train.csv', 'val': 'intell_residual_valid.csv', 'test': 'intell_residual_test.csv'}
filename_final = filename_res
sample_size = 'site16_allimages'
batch_size = 8
onlyt1 = False
t1_mean = 0.35196779465675354 
t2_mean = 0.5694633522033692
t1_std = 0.8948413240464094
t2_std = 1.2991791534423829

In [4]:
train_df, val_df, test_df, norm_dict = return_csv(path_csv, filename_final, False)

In [5]:
cat_cols = {'female': 2, 'race.ethnicity': 5, 'high.educ_group': 4, 'income_group': 8, 'married': 6}
num_cols = [x for x in list(val_df.columns) if '_norm' in x]

In [40]:
output = []
for col in num_cols:
  mean = test_df[col].mean()
  mse_norm = np.mean(np.square(test_df[col]-mean))
  output.append([col, mse_norm])

for col in list(cat_cols.keys()):
  mean = test_df[col].value_counts().idxmax()
  mse_norm = np.mean(test_df[col]==mean)
  output.append([col, mse_norm])

In [41]:
df = pd.DataFrame(output)
df.columns = ['colname', 'mse']

In [42]:
df_base = df.copy()

In [43]:
df_add = ['runAllImages64_SimpleCNN_T1T2_multitask_test.csv', 'runAllImages64_DenseNet_T1T2_multitask_test.csv', 'runAllImages64_SimpleCNN_T1_multitask_test.csv']

In [44]:
for df_name in df_add:
    df = pd.read_csv('./output/' + df_name)
    df.columns = [df_name + str(x) for x in df.columns]
    df_base = pd.merge(df_base, df, left_on='colname', right_on=df_name+'name',how='left')

In [49]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x or 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = range(0,aa.shape[1])
aa[[0,1] + [x for x in range(2,aa.shape[1],2)]]

,0,1,2,4,6
0,BMI_norm,0.930882,0.224202,0.213756,0.237303
1,age_norm,1.005556,1.041198,0.886657,0.972135
2,vol_norm,0.986314,0.397335,0.043417,0.141944
3,weight_norm,0.978025,0.178148,0.128281,0.192569
4,height_norm,1.084821,0.744058,0.740832,0.624957
5,nihtbx_fluidcomp_uncorrected_norm,0.992199,1.177742,0.986537,1.016971
6,nihtbx_cryst_uncorrected_norm,1.029535,1.206341,1.026766,1.198404
7,nihtbx_pattern_uncorrected_norm,0.969806,0.982501,0.982749,0.993296
8,nihtbx_picture_uncorrected_norm,0.916866,0.938528,0.920536,0.934019
9,nihtbx_list_uncorrected_norm,1.048735,1.195823,1.063641,1.146397


In [50]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x or 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = range(0,aa.shape[1])
aa[[0,1] + [x+1 for x in range(2,aa.shape[1],2)]]

,0,1,3,5,7
0,BMI_norm,0.930882,3.941825,3.758173,4.172169
1,age_norm,1.005556,58.241919,49.597262,54.378690
2,vol_norm,0.986314,0.398687,0.043565,0.142427
3,weight_norm,0.978025,96.896373,69.772881,104.739823
4,height_norm,1.084821,7.395848,7.363771,6.211992
5,nihtbx_fluidcomp_uncorrected_norm,0.992199,0.883379,0.739964,0.762792
6,nihtbx_cryst_uncorrected_norm,1.029535,0.742502,0.631973,0.737616
7,nihtbx_pattern_uncorrected_norm,0.969806,0.884211,0.884434,0.893926
8,nihtbx_picture_uncorrected_norm,0.916866,0.860987,0.844480,0.856849
9,nihtbx_list_uncorrected_norm,1.048735,0.946903,0.842236,0.907766


In [43]:
df_base.to_csv('output.csv')

In [6]:
train_df, val_df, test_df, norm_dict = return_csv(path_csv, filename_final, False)

In [7]:
train_df['abcd_site'].value_counts()

16    605
4     386
20    384
13    331
14    324
10    316
3     314
2     311
6     308
21    301
17    289
12    288
19    246
9     207
11    207
5     185
15    182
7     180
1     171
18    163
8     116
22     17
Name: abcd_site, dtype: int64

In [8]:
train_df = train_df[train_df['abcd_site']==16]
val_df = val_df[val_df['abcd_site']==16]
test_df = test_df[test_df['abcd_site']==16]

In [9]:
cat_cols = {'female': 2, 'race.ethnicity': 5, 'high.educ_group': 4, 'income_group': 8, 'married': 6}
num_cols = [x for x in list(val_df.columns) if '_norm' in x]

In [10]:
output = []
for col in num_cols:
  mean = test_df[col].mean()
  mse_norm = np.mean(np.square(test_df[col]-mean))
  output.append([col, mse_norm])

for col in list(cat_cols.keys()):
  mean = test_df[col].value_counts().idxmax()
  mse_norm = np.mean(test_df[col]==mean)
  output.append([col, mse_norm])

In [11]:
df = pd.DataFrame(output)
df.columns = ['colname', 'mse']

In [12]:
df_base = df.copy()

In [13]:
df_add = ['runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csv', 
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20emultitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20emultitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20e_onlyIntel_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20e_SimpleCNN_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_onlyIntel_20e_onlyIntel_SimpleCNN_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_SimpleCNN_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T2_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_def_multitask_test.csv',
          'runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csv',
          ]

In [14]:
for df_name in df_add:
    df = pd.read_csv('./output/' + df_name)
    df.columns = [df_name + str(x) for x in df.columns]
    df_base = pd.merge(df_base, df, left_on='colname', right_on=df_name+'name',how='left')

In [15]:
df_base.head()

,colname,mse,runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csvUnnamed: 0,runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csvname,runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csvbest_loss_test,runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csvbest_acc_test,runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csvbest_loss_val,runAllImages64_DenseNet_T1T2_site16_norm_multitask_test.csvbest_acc_val,runAllImages64_DenseNet_T1T2_site16_onlyIntel_multitask_test.csvUnnamed: 0,runAllImages64_DenseNet_T1T2_site16_onlyIntel_multitask_test.csvname,...,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_def_multitask_test.csvbest_loss_test,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_def_multitask_test.csvbest_acc_test,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_def_multitask_test.csvbest_loss_val,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_def_multitask_test.csvbest_acc_val,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csvUnnamed: 0,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csvname,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csvbest_loss_test,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csvbest_acc_test,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csvbest_loss_val,runAllImages64_DenseNet_T1T2_site16_norm_20e_norm_T1T2def_multitask_test.csvbest_acc_val
0,BMI_norm,0.927937,5,BMI_norm,0.463224,8.144222,0.331028,5.820014,NaN,NaN,...,0.947638,16.661003,0.726132,12.766572,5,BMI_norm,0.882645,15.518318,0.637507,11.208405
1,age_norm,1.095604,6,age_norm,1.062673,59.443111,1.135046,63.491507,NaN,NaN,...,1.181736,66.103231,1.301024,72.775903,6,age_norm,1.123761,62.860244,1.225986,68.578469
2,vol_norm,0.583598,7,vol_norm,0.413611,0.415018,0.372686,0.373954,NaN,NaN,...,0.582433,0.584415,0.747358,0.749901,7,vol_norm,0.251245,0.252099,0.297139,0.298150
3,weight_norm,1.095977,8,weight_norm,0.495430,269.468494,0.359302,195.427670,NaN,NaN,...,1.161725,631.871798,0.824611,448.512839,8,weight_norm,0.259650,141.225698,0.208632,113.476530
4,height_norm,0.859132,9,height_norm,0.619130,6.154068,0.500924,4.979118,NaN,NaN,...,0.899410,8.940024,0.829024,8.240392,9,height_norm,0.613781,6.100909,0.624105,6.203522


In [18]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x or 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = range(0,aa.shape[1])
aa[[0,1] + [x for x in range(2,aa.shape[1],2)]].to_csv('ana1.csv')

In [19]:
add_col = [x for x in df_base.columns if 'best_loss_test' in x or 'best_acc_test' in x]
aa = df_base[['colname', 'mse'] + add_col]
aa.columns = range(0,aa.shape[1])
aa[[0,1] + [x+1 for x in range(2,aa.shape[1],2)]].to_csv('ana2.csv')

In [17]:
df_base.to_csv('output_site16.csv')

In [43]:
test_df['female'].value_counts()

0    66
1    56
Name: female, dtype: int64

In [57]:
66/(66+56)

0.5409836065573771

In [83]:
from tf_explain.core.smoothgrad import SmoothGrad